## A Restaurant Recommendation System based on Yelp Information ##

**Project Goal**: Create an NYC restaurant information database and use the database to recommend users restaurants which match with their preferences.

Users can input features that indicate their preferences and our machine can output top 5 restaurants from as our recommendation for the user.

**Steps**:

1. We use "Yelp Restaurant Info Scraper.ipynb" to get search urls and scrape information for restaurants with different cuisine type. The script iterates through all locations in NYC. We ran the script on our local machines.

2. After getting the csv for each cuisine type, we use "Dataframe Cleaner.ipynb" to clean and merge all the csv files as a final dataframe. We use this final dataframe (final_df.csv) as our database.



In [1]:
from bs4 import BeautifulSoup
import re
from threading import Thread
import urllib
import pandas as pd
import urllib.request
import time
from random import randint

In [ ]:
opener = urllib.request.build_opener()
# IE 9 proved to be the most successful
opener.addheaders = [('User-agent', 'IE 9/Windows: Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)')]
urllib.request.install_opener(opener)

In [ ]:
# Function that will do the scraping job from yelp
def scrape(ur):

    with urllib.request.urlopen(ur) as url:
        html = url.read()
    soup = BeautifulSoup(html,"lxml")
    retaurant_name = soup.find('h1')
    
    # create a dictionary business info for storing key business features 
    business_info = {}
    business_info['restaurant_name']= str(retaurant_name.text.strip().rstrip())
    
    if soup.find('span',itemprop="streetAddress") != None:
        retaurant_address = soup.find('span',itemprop="streetAddress")
        business_info['retaurant_address'] = str(retaurant_address.text.strip().rstrip())
    
    if soup.find('span',itemprop="postalCode") != None:
        restaurant_zipcode = soup.find('span',itemprop="postalCode")
        business_info['restaurant_zipcode'] = str(restaurant_zipcode.text.strip().rstrip())
    
    if soup.find('span',itemprop="reviewCount") != None:
        restaurant_reviewcount = soup.find('span',itemprop="reviewCount")
        business_info['restaurant_reviewcount'] = str(restaurant_reviewcount.text.strip().rstrip())
   
    if soup.find(itemprop="ratingValue") != None:
        business_info['restaurant_rating'] = soup.find(itemprop="ratingValue").get("content")

    if soup.find('span', {'class': 'neighborhood-str-list'}) != None:
        neighborhood = soup.find('span', {'class': 'neighborhood-str-list'})
        business_info['restaurant_neighobrhood'] = str(neighborhood.text.strip().rstrip())
   
    if soup.find('dd',{'class':"nowrap health-score-description"}) != None:
        hygiene_score = soup.find('dd',{'class':"nowrap health-score-description"})
        business_info['Hygiene_score'] = str(hygiene_score.text.strip().rstrip())
        
    if soup.find('dd', {'class':"nowrap price-description"}) != None:
        price_range = soup.find('dd', {'class':"nowrap price-description"})
        business_info['price_range'] = str(price_range.text.strip().rstrip())
   
    if soup.find('div',{'class':'short-def-list'}) != None:
        for i in soup.find('div',{'class':'short-def-list'}).findAll('dl'):
            key = i.find('dt').text.strip().rstrip()
            value = i.find('dd').text.strip().rstrip()
            business_info[str(key)]=str(value)
    
    if soup.find(property="place:location:latitude") != None:
        business_info['latitude'] = soup.find(property="place:location:latitude").get("content")

    if soup.find(property="place:location:longitude") != None:
        business_info['longitude'] = soup.find(property="place:location:longitude").get("content")  
    
    business_info['Category']= ''
    if soup.find('span',{'class':'category-str-list'}) != None:
        for i in soup.find('span',{'class':'category-str-list'}).findAll('a'):
            business_info['Category'] += (str(i.text.strip().rstrip())+'; ')
                
    return business_info

In [ ]:
# Example for the scrape function
d = scrape('https://www.yelp.com/biz/blue-ribbon-sushi-new-york?osq=blue+ribbon')
print(d)

In [ ]:
# List of yelp urls to scrape
def get_urls_from_search(term, location, num):
    
    term = term.replace(' ','+')
    location = location.replace(' ','+')
    query = 'https://www.yelp.com/search?find_desc='+term+'&find_loc='+location+'&start='+str(num*10)
    with urllib.request.urlopen(query) as url:
        contents = url.read()
    #contents = urllib.urlopen(query).read()
    soup = BeautifulSoup(contents, "html.parser")
    #print(soup)
    business_url = []
    for result in soup.findAll('a',{'class':'biz-name js-analytics-click'}):
        business_url.append("http://www.yelp.com" + result['href'])
    return business_url


In [ ]:
# List of all locations
# 'Alphabet_City','Battery_Park','Chelsea','Chinatown','Civic_Center','East_Harlem','East_Village','Financial_District','Flatiron','Gramercy','Greenwich_Village','Harlem','Hell\'s_Kitchen','Inwood','Kips_Bay','Koreatown','Little_Italy','Lower_East_Side','Manhattan_Valley','Marble_Hill','Meatpacking_District','Midtown_East','Midtown_West','Morningside_Heights','Murray_Hill','NoHo','Nolita','Roosevelt_Island','SoHo','South_Street_Seaport','South_Village','Stuyvesant_Town','Theater_District','TriBeCa','Two_Bridges','Union_Square','Upper_East_Side','Upper_West_Side','Washington_Heights','West_Village', 'Yorkville'

# Yorkville entirely encompassed by Upper East Side
# Theater District entirely encompassed by Midtown West

searchLocations = ['Alphabet_City','Battery_Park','Chelsea','Chinatown','Civic_Center','East_Harlem','East_Village','Financial_District','Flatiron','Gramercy','Greenwich_Village','Harlem','Hell\'s_Kitchen','Inwood','Kips_Bay','Koreatown','Little_Italy','Lower_East_Side','Manhattan_Valley','Marble_Hill','Meatpacking_District','Midtown_East','Midtown_West','Morningside_Heights','Murray_Hill','NoHo','Nolita','Roosevelt_Island','SoHo','South_Street_Seaport','South_Village','Stuyvesant_Town','TriBeCa','Two_Bridges','Union_Square','Upper_East_Side','Upper_West_Side','Washington_Heights','West_Village']

#test
#searchLocations = ['East_Village', 'Upper_West_Side', 'Chelsea'] 

In [ ]:
max_num = 5
urls_set = []
for i, loc in enumerate(searchLocations):
    # now run for loop with fix location and food type and append urls 
    # page ranking is based on relevance ranked by yelp
    for num in range(0,max_num):
        urls = get_urls_from_search("Japanese restaurants",loc, num)
        urls = urls[1:] # 0th link is irrelavant
        # len(urls)=0 if the starting page number exceed the maximum possible
        if (len(urls) ==0):
            break
        else:
            for i in range(0,len(urls)-1):
                urls_set.append(urls[i])
                
    # Delays to help reduce queries and reduce the possibility of IP Ban            
    time.sleep(5)
    #convert the urls from list to csv file for each location
    #pd_urls_set = pd.DataFrame(urls_set)
    #pd_urls_set.to_csv("urls_set_{0}.csv".format(loc))

In [ ]:
print(len(urls_set))
urls_set

In [ ]:
info = {} # create a dictionary for srestaurant info
for u in urls_set:
    url_dict=scrape(u)
    for key,value in url_dict.items():
        info.setdefault(key,[]).append(value)

In [ ]:
#header

In [ ]:
header=['restaurant_name', 'retaurant_address', 'restaurant_zipcode', 'restaurant_reviewcount', 'restaurant_rating', 'Hygiene_score', 'price_range', 'Takes Reservations', 'Delivery', 'Take-out', 'Accepts Credit Cards', 'Accepts Apple Pay', 'Accepts Google Pay', 'Accepts Bitcoin', 'Good For', 'Parking', 'Bike Parking', 'Wheelchair Accessible', 'Good for Kids', 'Good for Groups', 'Attire', 'Ambience', 'Noise Level', 'Alcohol', 'Happy Hour', 'Outdoor Seating', 'Wi-Fi', 'Has TV', 'Dogs Allowed', 'Waiter Service', 'Caters', 'Gender Neutral Restrooms', 'Has Dairy-free Options', 'Liked by Vegetarians', 'Liked by Vegans', 'Good For Dancing', 'Best Nights', 'Good for Working', 'Has Pool Table', 'Open to All', 'Coat Check', 'Has Soy-free Options', 'Has Gluten-free Options', 'Smoking', 'Has Kosher Options', 'Offers Military Discount']
info={}
for u in urls_set:
    url_dict=scrape(u)
    for i in header:
        if i in url_dict.keys():
            info.setdefault(i,[]).append(url_dict[i])
        else:
            info.setdefault(i,[]).append('NA')
    # Delays to help reduce queries and reduce the possibility of IP Ban
    time.sleep(5)

In [ ]:
final_df_sample=pd.DataFrame(info)
#final_df_sample

In [ ]:
final_df_sample.to_csv('sample japnese.csv') # save to csv

In [ ]:
len(final_df)
final_df.head()

In [ ]:
max_num = 5
urls_set = []
for i, loc in enumerate(searchLocations):
    # now run for loop with fix location and food type and append urls 
    # page ranking is based on relevance ranked by yelp
    for num in range(0,max_num):
        urls = get_urls_from_search("Chinese restaurants",loc, num)
        urls = urls[1:] # 0th link is irrelavant
        # len(urls)=0 if the starting page number exceed the maximum possible
        if (len(urls) ==0):
            break
        else:
            for i in range(0,len(urls)-1):
                urls_set.append(urls[i])
                
    time.sleep(5)
    #convert the urls from list to csv file for each location
    #pd_urls_set = pd.DataFrame(urls_set)
    #pd_urls_set.to_csv("urls_set_{0}.csv".format(loc))

In [ ]:
len(urls_set)

In [ ]:
header=['restaurant_name', 'retaurant_address', 'restaurant_zipcode', 'restaurant_reviewcount', 'restaurant_rating', 'restaurant_neighobrhood', 'Hygiene_score', 'price_range', 'Liked by Vegetarians', 'Takes Reservations', 'Delivery', 'Take-out', 'Accepts Credit Cards', 'Accepts Bitcoin', 'Parking', 'Bike Parking', 'Wheelchair Accessible', 'Good for Kids', 'Good for Groups', 'Attire', 'Noise Level', 'Alcohol', 'Happy Hour', 'Outdoor Seating', 'Wi-Fi', 'Has TV', 'Dogs Allowed', 'Waiter Service', 'Caters', 'Category', 'Has Soy-free Options', 'Has Dairy-free Options', 'Liked by Vegans', 'Has Gluten-free Options', 'Good For', 'Ambience', 'Gender Neutral Restrooms']
info={}
for u in urls_set[:30]:
    url_dict=scrape(u)
    for i in header:
        if i in url_dict.keys():
            info.setdefault(i,[]).append(url_dict[i])
        else:
            info.setdefault(i,[]).append('NA')
     # Delays to help reduce queries and reduce the possibility of IP Ban
    time.sleep(2)

In [ ]:
df = pd.DataFrame(info)
df.to_csv('sample chinese.csv') 

In [ ]:
df.head()

In [ ]:
print('Please enter a restaurant name below, if you have no idea about it, just click return')
name=input("restaurant name: ")

if name=='':
    #return the original csv
else:
    #return the selceted csv

In [ ]:
print('Please enter a restaurant style below, if you have no idea about it, just click return')
style = input("style: ")

In [ ]:
print('Are you going to eat Lunch, Dinner or Breakfast?, please input Lunch or Dinner, if you have no idea about it, just click return')
mealkind = input("Lunch, Dinner or Breakfast :")

In [ ]:
print('Want to find a place to drink alcohol? Please enter: Full Bar, Beer&Wine Only or No below')
alcohol = input("alcohol:")

In [ ]:
print("enter your current location: ")
zipcode = input("location:")

In [ ]:
# We want to output the restaurants with highest Yelp ratings and good hygiene for the users

In [ ]:
hehe = pd.read_csv('final_df.csv', encoding='latin1')

'''
for item in hehe['restaurant_neighborhood'].tolist():
    if isinstance(item, str):
        print(item)
'''

hehe

In [2]:
def ask_for_name(df):
    
    nl = []
    while True:
        print('Please enter a restaurant name below, if you have no idea about it, just click return\n')
        name=input("Restaurant name: \n")
        
        if name == '':
            return df
        
        for item in df['restaurant_name'].tolist():
            if isinstance(item, str):
                if re.search(name.lower(), item.lower()):
                    nl.append(item)
        
        if len(nl) == 0:
            print('Please enter a valid name.\n')
            continue
        else:
            break
    return df.loc[df['restaurant_name'].isin(nl)]

def ask_for_category(df):
    
    cl = []
    while True:
        print('Please enter a restaurant category below, if you have no idea about it, just click return\n')
        cat = input("Restaurant category: \n")
        
        if cat == '':
            return df
        
        for item in df['Category'].tolist():
            if isinstance(item, str):
                if re.search(cat.lower(), item.lower()):
                    cl.append(item)
        
        if len(cl) == 0:
            print('Please enter a valid category.\n')
            continue
        else:
            break
    return df.loc[df['Category'].isin(cl)]

def ask_for_neighbor(df):
    
    nl = []
    while True:
        print('Please enter a restaurant neighborhood below, if you have no idea about it, just click return\n')
        neighbor = input("Restaurant neighborhood: \n")
        
        if neighbor == '':
            return df
        
        for item in df['restaurant_neighborhood'].tolist():
            if isinstance(item, str):
                if re.search(neighbor.lower(), item.lower()):
                    nl.append(item)
                
        if len(nl) == 0:
            print("Please enter a valid neighborhood. \n")
            continue
        else:
            break
    
    return df.loc[df['restaurant_neighborhood'].isin(nl)]

def ask_for_rating(df):
    
    
    while True:
        try:
            print("Please enter a range of your expected rating below, if you have no idea about it, just click return\n")
            lb = input("Please enter a lower bound of rating range you are looking for :\n")
            ub = input("Please enter an upper bound of rating range you are looking for :\n")
        
            if lb == '' and ub == '':
                return df
        
            if lb == '':
                lb = '0'
        
            if ub == '':
                ub = '5'
            
            if float(lb) > float(ub):
                print('Lower bound has to be smaller than the upper bound.\n')
                continue
        
            if float(lb) < 0 or float(ub) > 5:
                print('The range of rating has to be between 0 and 5.\n')
                continue
            
            return df.loc[(df['restaurant_rating'] >= float(lb)) & (df['restaurant_rating'] <= float(ub))]
        
        except:
        
            print('Please provide a valid input\n')
            continue

# Credit to ZiYuan Wang
def ask_for_party(df):
    
    while True:
        print("Good for groups / parties?\n")
        PARTY = input("Please enter Y/N\n")
        
        if PARTY == 'Y':
            A = 'Yes'
        
        elif PARTY =='N':
            A='No'
            
        else: 
            continue
        
        return df.loc[df['Good for Groups']==A]
    
def ask_for_price(df):
    
    pl = []
    while True:
        try:
            print("Please enter a range of your expected price below, if you have no idea about it, just click return\n")
            lb = input("Please enter a lower bound of price range you are looking for :\n")
            ub = input("Please enter an upper bound of price range you are looking for :\n")
            if lb == '' and ub == '':
                return df
        
            for item in df['price_range'].tolist():
                if isinstance(item, str):
                    if re.search('^\$\d+-\d+$', item):
                        i1 = int(item[1:item.find('-')])
                        i2 = int(item[item.find('-')+1:])
                    
                        if float(lb) <= i1 or float(ub) >= i2:
                            pl.append(item)
                    elif 'Under' in item:
                    
                        if int(item[item.find('$')+1:]) >= float(lb):
                            pl.append(item)
                        
                    elif 'Above' in item:
                    
                        if int(item[item.find('$')+1:]) <= float(ub):
                            pl.append(item)
                        
            if len(pl) == 0:
                print("Unfortunately, there are no restaurants falling into your desired price range. Please try some other price range\n")
                continue
            else:
                
                return df.loc[df['price_range'].isin(pl)]
    
        except:
        
            print('Please provide a valid input\n')
            continue

In [ ]:
#ha = ask_for_category(pd.read_csv('/Users/haodi_liu/Documents/GitHub/Are-you-hungry-team-13/final_df.csv', encoding='latin1'))
#ask_for_category(ha)

ask_for_neighbor(pd.read_csv('/Users/haodi_liu/Documents/GitHub/Are-you-hungry-team-13/final_df.csv', encoding='latin1'))


In [9]:
global_df = pd.read_csv('/Users/haodi_liu/Documents/GitHub/Are-you-hungry-team-13/final_df.csv', encoding='latin1')
done = False

while done == False:
    
    global_df = ask_for_name(global_df)
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
    
    while True:
        global_df = ask_for_category(global_df)
        length = len(global_df['restaurant_name'].tolist())
        print(f"There are {length} restaurants filtered out so far\n")
        con = input('Do you want to keep filtering based on Category? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
        if con == '0':
            break
        
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
        
    while True:
        global_df = ask_for_neighbor(global_df)
        length = len(global_df['restaurant_name'].tolist())
        print(f"There are {length} restaurants filtered out so far\n")
        con = input('Do you want to keep filtering based on Neighborhood? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
        if con == '0':
            break
    
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
    
    global_df = ask_for_rating(global_df)
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
    
    global_df = ask_for_price(global_df)
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
        
    global_df = ask_for_party(global_df)
    length = len(global_df['restaurant_name'].tolist())
    print(f"There are {length} restaurants filtered out so far\n")
    con = input('Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.\n')
    if con == '0':
        break
        
    done = True
    
a4 = global_df
global_df

Please enter a restaurant name below, if you have no idea about it, just click return

Restaurant name: 

There are 1871 restaurants filtered out so far

Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.
C
Please enter a restaurant category below, if you have no idea about it, just click return

Restaurant category: 
Chinese
There are 449 restaurants filtered out so far

Do you want to keep filtering based on Category? Enter 0 if you are done with searching. Otherwise, enter anything else.
Chinese
Please enter a restaurant category below, if you have no idea about it, just click return

Restaurant category: 
Chinese
There are 449 restaurants filtered out so far

Do you want to keep filtering based on Category? Enter 0 if you are done with searching. Otherwise, enter anything else.
0
There are 449 restaurants filtered out so far

Do you want to keep searching? Enter 0 if you are done with searching. Otherwise, enter anything else.
as


,Unnamed: 0,Accepts Bitcoin,Accepts Credit Cards,Alcohol,Ambience,Attire,Bike Parking,Category,Caters,Delivery,...,Wheelchair Accessible,Wi-Fi,price_range,restaurant_name,restaurant_neighborhood,restaurant_rating,restaurant_reviewcount,restaurant_zipcode,retaurant_address,Source
74,2,NaN,Yes,No,Casual,Casual,No,Chinese; Noodles;,No,Yes,...,NaN,No,Under $10,Noodle Village,"Chinatown, Civic Center",4.0,877.0,10013,13 Mott St,Chinese
76,4,NaN,Yes,Beer & Wine Only,Casual,Casual,Yes,Chinese; Seafood; Noodles;,No,Yes,...,NaN,No,$11-30,Hop Lee Restaurant,"Chinatown, Civic Center",4.0,256.0,10013,16 Mott St,Chinese
80,8,No,Yes,No,Casual,Casual,No,Chinese; Kosher; Vegan;,Yes,Yes,...,Yes,No,$11-30,Buddha Bodai Kosher Vegetarian Restaurant,"Chinatown, Civic Center",4.0,881.0,10013,5 Mott St,Chinese
89,17,NaN,No,No,Casual,Casual,Yes,Taiwanese; Chinese; Noodles;,No,Yes,...,NaN,No,Under $10,Taiwan Pork Chop House,"Chinatown, Civic Center",4.0,349.0,10013,3 Doyers St,Chinese
92,20,No,Yes,No,Casual,Casual,Yes,Taiwanese; Bubble Tea; Chinese;,Yes,Yes,...,No,No,Under $10,Taiwan Bear House,"Chinatown, Civic Center",4.0,268.0,10013,11 Pell St,Chinese
95,23,No,Yes,No,Casual,Casual,No,Asian Fusion; Malaysian; Chinese;,Yes,Yes,...,Yes,No,Under $10,Asian Spicy Curry Pop-up,"Chinatown, Civic Center",4.5,123.0,10013,46 Mulberry St,Chinese
695,43,No,Yes,No,Casual,Casual,No,Asian Fusion; Malaysian; Chinese;,Yes,Yes,...,Yes,No,Under $10,Asian Spicy Curry Pop-up,"Chinatown, Civic Center",4.5,123.0,10013.0,46 Mulberry St,American
705,53,NaN,No,No,Casual,Casual,Yes,Taiwanese; Chinese; Noodles;,No,Yes,...,NaN,No,Under $10,Taiwan Pork Chop House,"Chinatown, Civic Center",4.0,351.0,10013.0,3 Doyers St,American
708,56,No,Yes,No,Casual,Casual,Yes,Taiwanese; Bubble Tea; Chinese;,Yes,Yes,...,No,No,Under $10,Taiwan Bear House,"Chinatown, Civic Center",4.0,268.0,10013.0,11 Pell St,American
733,89,NaN,Yes,No,Casual,Casual,No,Chinese; Noodles;,No,Yes,...,NaN,No,Under $10,Noodle Village,"Chinatown, Civic Center",4.0,876.0,10013.0,13 Mott St,American


In [ ]:
hehe.colums.get_loc("Reservation")

In [ ]:
hehe.iloc[5,26]

In [ ]:
def get_out_put_1(d,f):
    print(f'''###Recommendation{i+1}###''')
    print(f'''restaurant name:                {d.iloc[i,32]}''')
    print(f'''restaurant address:             {d.iloc[i,37]} New York, NY {d.iloc[0,36]}''')
    print(f'''price range:                        {d.iloc[i,31]}''') 
    print(f'''restaurant rating:                {d.iloc[i,34]}''')
    print(f'''restaurant hygiene rate:       {d.iloc[i,20]}''')
    print(f'''restaurant attire:                 {d.iloc[i,5]} ''' )
    print(f'''restaurant parking:              {d.iloc[i,25]}''')
    print(f'''restaurant reservable:          {d.iloc[i,26]}''')
    print(f'''restaurant ambience:           {d.iloc[i,4]}''')

In [ ]:
def get_out_put_2(d,c,g):
    print(f'''###Recommendation{i+1}###''')
    print(f'''restaurant name:                {d.iloc[a[i],32]}''')
    print(f'''restaurant rating:                {d.iloc[a[i],34]}''')
    print(f'''restaurant hygiene rate:       {d.iloc[a[i],20]}''')
    print(f'''restaurant attire:                 {d.iloc[a[i],5]}''' )
    print(f'''price range:                        {d.iloc[a[i],31]}''')
    print(f'''restaurant address:             {d.iloc[a[i],37]} New York, NY {d.iloc[a[i],36]} ''')
    print(f'''restaurant ambience:           {d.iloc[a[i],4]}''')
    print(f'''restaurant parking:              {d.iloc[a[i],25]}''')
    print(f'''restaurant reservable:          {d.iloc[a[i],26]}''')

In [ ]:
if a4.shape[0]<=5:
    for i in range(a4.shape[0]):
        get_out_put_1(a4,i)
        print('\n')
elif a4.shape[0]>5:
    a=[]
    list_1=[]
    for q in range(a4.shape[0]):
        list_1.append(a4.iloc[q,34])
    list_1s=sorted(list_1, reverse=True)
    for i in range(len(list_1)):
        if list_1s[0]==list_1[i]:
            a.append(i)
            break
    list_1[a[0]]=-1
    for i in range(len(list_1)):
        if list_1s[1]==list_1[i]:
            a.append(i)
            break
    list_1[a[1]]=-1
    for i in range(len(list_1)):
        if list_1s[2]==list_1[i]:
            a.append(i)
            break
    list_1[a[2]]=-1
    for i in range(len(list_1)):
        if list_1s[3]==list_1[i]:
            a.append(i)
            break
    list_1[a[3]]=-1
    for i in range(len(list_1)):
        if list_1s[4]==list_1[i]:
            a.append(i)
            break
    list_1[a[4]]=-1
    for i in range(len(a)):
        get_out_put_2(a4,i,a)
        print('\n')
elif a4.shape[0]<=0:
    print("Please search again, this time we do not have a restaurant satisfy your need. ")
